# Notebook 114: 正則化と最適化

## Regularization and Optimization

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第5章として、**正則化** と最適化の関係を学びます。

### 学習目標

1. **L1/L2正則化** の仕組みを理解する
2. **Weight Decay** と L2正則化の違いを理解する
3. **Dropout** の効果を理解する
4. **Batch Normalization** の正則化効果を理解する
5. 正則化と最適化の相互作用を学ぶ

### 前提知識

- Notebook 110-113 の内容

---

## 目次

1. [正則化の目的](#1-正則化の目的)
2. [L2正則化（Ridge）](#2-l2正則化ridge)
3. [L1正則化（Lasso）](#3-l1正則化lasso)
4. [Weight Decay vs L2正則化](#4-weight-decay-vs-l2正則化)
5. [Elastic Net](#5-elastic-net)
6. [Dropout](#6-dropout)
7. [Batch Normalization](#7-batch-normalization)
8. [演習問題](#8-演習問題)
9. [まとめと次のステップ](#9-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 正則化の目的

### 1.1 過学習（Overfitting）

モデルが訓練データに過度に適合し、新しいデータに対する性能が低下する現象。

### 1.2 正則化の役割

モデルの複雑さを制限することで、汎化性能を向上させます。

$$
L_{\text{regularized}} = L_{\text{data}} + \lambda \cdot R(\theta)
$$

- $L_{\text{data}}$: データに対する損失
- $R(\theta)$: 正則化項
- $\lambda$: 正則化の強さ

In [ ]:
# 過学習のデモンストレーション

np.random.seed(42)

# 真の関数: sin(x)
def true_function(x):
    return np.sin(x)

# データ生成
n_samples = 15
X_train = np.linspace(0, 2*np.pi, n_samples)
y_train = true_function(X_train) + np.random.randn(n_samples) * 0.3

# 多項式回帰
def polynomial_features(x, degree):
    return np.column_stack([x**i for i in range(degree + 1)])

def fit_polynomial(X, y, degree):
    X_poly = polynomial_features(X, degree)
    # 正規方程式で解く
    w = np.linalg.lstsq(X_poly, y, rcond=None)[0]
    return w

def predict_polynomial(x, w):
    degree = len(w) - 1
    X_poly = polynomial_features(x, degree)
    return X_poly @ w


# 異なる次数でフィット
degrees = [1, 3, 12]
X_test = np.linspace(0, 2*np.pi, 100)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, degree in zip(axes, degrees):
    w = fit_polynomial(X_train, y_train, degree)
    y_pred = predict_polynomial(X_test, w)
    
    ax.scatter(X_train, y_train, color='blue', label='訓練データ', zorder=5)
    ax.plot(X_test, true_function(X_test), 'g--', label='真の関数', linewidth=2)
    ax.plot(X_test, y_pred, 'r-', label=f'予測 (次数={degree})', linewidth=2)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'多項式次数 = {degree}')
    ax.legend()
    ax.set_ylim(-2, 2)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("- 次数1: アンダーフィット（モデルが単純すぎる）")
print("- 次数3: 適切なフィット")
print("- 次数12: オーバーフィット（訓練データに過度に適合）")

---

## 2. L2正則化（Ridge）

### 2.1 定義

$$
L_{\text{Ridge}} = L_{\text{data}} + \frac{\lambda}{2} \|\theta\|_2^2 = L_{\text{data}} + \frac{\lambda}{2} \sum_i \theta_i^2
$$

### 2.2 効果

- 重みを小さく保つ
- 極端に大きな重みを防ぐ
- 重みがゼロにはなりにくい

In [ ]:
def fit_polynomial_ridge(X, y, degree, lambda_reg):
    """L2正則化付き多項式回帰"""
    X_poly = polynomial_features(X, degree)
    n_features = X_poly.shape[1]
    
    # 正規方程式: (X^T X + λI)^-1 X^T y
    identity = np.eye(n_features)
    identity[0, 0] = 0  # バイアス項は正則化しない
    
    w = np.linalg.solve(X_poly.T @ X_poly + lambda_reg * identity, X_poly.T @ y)
    return w


# L2正則化の効果
degree = 12
lambdas = [0, 0.001, 0.1, 10]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, lambda_reg in zip(axes, lambdas):
    w = fit_polynomial_ridge(X_train, y_train, degree, lambda_reg)
    y_pred = predict_polynomial(X_test, w)
    
    ax.scatter(X_train, y_train, color='blue', zorder=5)
    ax.plot(X_test, true_function(X_test), 'g--', linewidth=2)
    ax.plot(X_test, y_pred, 'r-', linewidth=2)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'λ = {lambda_reg}')
    ax.set_ylim(-2, 2)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 重みの大きさを比較
print("【重みの L2ノルム】")
for lambda_reg in lambdas:
    w = fit_polynomial_ridge(X_train, y_train, degree, lambda_reg)
    norm = np.linalg.norm(w)
    print(f"λ = {lambda_reg:5.3f}: ||w||₂ = {norm:.4f}")

### 2.3 L2正則化の幾何学的理解

In [ ]:
# L2正則化の幾何学的解釈

# データ損失（楕円）
def data_loss(w1, w2):
    return (w1 - 3)**2 + 0.5 * (w2 - 2)**2

# L2正則化項（円）
def l2_reg(w1, w2):
    return w1**2 + w2**2

w1 = np.linspace(-1, 5, 200)
w2 = np.linspace(-1, 4, 200)
W1, W2 = np.meshgrid(w1, w2)

Z_data = data_loss(W1, W2)
Z_l2 = l2_reg(W1, W2)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# データ損失
axes[0].contour(W1, W2, Z_data, levels=20, cmap='Blues')
axes[0].scatter([3], [2], color='red', s=100, marker='*', label='最小点')
axes[0].set_xlabel('w₁')
axes[0].set_ylabel('w₂')
axes[0].set_title('データ損失 L(w)')
axes[0].legend()
axes[0].set_aspect('equal')

# L2正則化項
axes[1].contour(W1, W2, Z_l2, levels=20, cmap='Greens')
theta = np.linspace(0, 2*np.pi, 100)
for r in [1, 2, 3]:
    axes[1].plot(r*np.cos(theta), r*np.sin(theta), 'g-', alpha=0.5)
axes[1].scatter([0], [0], color='green', s=100, marker='*')
axes[1].set_xlabel('w₁')
axes[1].set_ylabel('w₂')
axes[1].set_title('L2正則化 ||w||²')
axes[1].set_aspect('equal')

# 合計損失
lambda_reg = 0.5
Z_total = Z_data + lambda_reg * Z_l2

# 最小点を数値的に求める
min_idx = np.unravel_index(np.argmin(Z_total), Z_total.shape)
w1_opt = w1[min_idx[1]]
w2_opt = w2[min_idx[0]]

axes[2].contour(W1, W2, Z_total, levels=20, cmap='Purples')
axes[2].scatter([3], [2], color='blue', s=100, marker='*', label='正則化なし')
axes[2].scatter([w1_opt], [w2_opt], color='red', s=100, marker='*', label='正則化あり')
axes[2].set_xlabel('w₁')
axes[2].set_ylabel('w₂')
axes[2].set_title(f'合計損失 (λ={lambda_reg})')
axes[2].legend()
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

print("【幾何学的解釈】")
print("- L2正則化は原点に向かって重みを引き付ける")
print(f"- 正則化なしの最適値: (3.0, 2.0)")
print(f"- 正則化ありの最適値: ({w1_opt:.2f}, {w2_opt:.2f})")

---

## 3. L1正則化（Lasso）

### 3.1 定義

$$
L_{\text{Lasso}} = L_{\text{data}} + \lambda \|\theta\|_1 = L_{\text{data}} + \lambda \sum_i |\theta_i|
$$

### 3.2 効果

- **スパース性**: 多くの重みを正確にゼロにする
- **特徴選択**: 重要でない特徴を自動的に削除

In [ ]:
# L1正則化の幾何学的解釈

# L1正則化項（ダイヤモンド）
def l1_reg(w1, w2):
    return np.abs(w1) + np.abs(w2)

Z_l1 = l1_reg(W1, W2)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# L2 vs L1
# L2
axes[0].contour(W1, W2, Z_data, levels=10, cmap='Blues', alpha=0.5)
for r in [0.5, 1, 1.5, 2, 2.5]:
    axes[0].plot(r*np.cos(theta), r*np.sin(theta), 'g-', alpha=0.5)
axes[0].scatter([3], [2], color='blue', s=100, marker='o', label='正則化なし')
# 最適点（L2）
axes[0].scatter([1.5], [1.3], color='red', s=100, marker='*', label='L2正則化')
axes[0].set_xlabel('w₁')
axes[0].set_ylabel('w₂')
axes[0].set_title('L2正則化: 等高線は円')
axes[0].set_xlim(-1, 4)
axes[0].set_ylim(-1, 3)
axes[0].legend()
axes[0].set_aspect('equal')

# L1
axes[1].contour(W1, W2, Z_data, levels=10, cmap='Blues', alpha=0.5)
# ダイヤモンドを描画
for r in [0.5, 1, 1.5, 2, 2.5]:
    diamond = np.array([[r, 0], [0, r], [-r, 0], [0, -r], [r, 0]])
    axes[1].plot(diamond[:, 0], diamond[:, 1], 'g-', alpha=0.5)
axes[1].scatter([3], [2], color='blue', s=100, marker='o', label='正則化なし')
# L1の最適点（角に接触）
axes[1].scatter([2], [0], color='red', s=100, marker='*', label='L1正則化')
axes[1].set_xlabel('w₁')
axes[1].set_ylabel('w₂')
axes[1].set_title('L1正則化: 等高線はダイヤモンド')
axes[1].set_xlim(-1, 4)
axes[1].set_ylim(-1, 3)
axes[1].legend()
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

print("【L1 vs L2】")
print("- L2: 最適点は通常、軸上にない → 重みはゼロにならない")
print("- L1: 最適点は角（軸上）に接触しやすい → 重みがゼロになる（スパース）")

In [ ]:
# L1正則化による特徴選択のデモ

def soft_threshold(x, threshold):
    """近接勾配法のL1に対する近接演算子"""
    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)


def lasso_coordinate_descent(X, y, lambda_reg, max_iter=1000, tol=1e-6):
    """Lasso回帰（座標降下法）"""
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    
    for _ in range(max_iter):
        w_old = w.copy()
        
        for j in range(n_features):
            # 残差の計算（j番目の特徴を除く）
            residual = y - X @ w + X[:, j] * w[j]
            # j番目の重みを更新
            rho = X[:, j] @ residual
            w[j] = soft_threshold(rho, lambda_reg * n_samples) / (X[:, j] @ X[:, j])
        
        if np.linalg.norm(w - w_old) < tol:
            break
    
    return w


# テスト: 高次元データでスパース性を確認
np.random.seed(42)
n_samples = 100
n_features = 20
n_informative = 5  # 実際に重要な特徴は5個だけ

# データ生成
X_sparse = np.random.randn(n_samples, n_features)
true_w = np.zeros(n_features)
true_w[:n_informative] = np.random.randn(n_informative) * 2  # 最初の5個だけ非ゼロ
y_sparse = X_sparse @ true_w + np.random.randn(n_samples) * 0.5

# L1正則化で回帰
lambdas = [0.01, 0.1, 0.5]

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, lambda_reg in zip(axes, lambdas):
    w_lasso = lasso_coordinate_descent(X_sparse, y_sparse, lambda_reg)
    
    ax.bar(range(n_features), true_w, alpha=0.5, label='真の重み')
    ax.bar(range(n_features), w_lasso, alpha=0.5, label='推定重み')
    ax.set_xlabel('特徴インデックス')
    ax.set_ylabel('重み')
    ax.set_title(f'L1正則化 (λ={lambda_reg})\n非ゼロ: {np.sum(np.abs(w_lasso) > 0.01)}/{n_features}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. Weight Decay vs L2正則化

### 4.1 SGDでは同等

**L2正則化**:
$$
\theta_{t+1} = \theta_t - \eta (\nabla L + \lambda \theta_t) = (1 - \eta\lambda)\theta_t - \eta \nabla L
$$

**Weight Decay**:
$$
\theta_{t+1} = (1 - \eta\lambda)\theta_t - \eta \nabla L
$$

→ 同じ！

### 4.2 Adamでは異なる

In [ ]:
# Adam + L2正則化 vs AdamW の違い

print("【Adam + L2正則化】")
print("")
print("1. 勾配を計算: g = ∇L + λθ")
print("2. 1次モーメント: m = β₁m + (1-β₁)g")
print("3. 2次モーメント: v = β₂v + (1-β₂)g²")
print("4. 更新: θ = θ - η * m̂ / √v̂")
print("")
print("→ 正則化項 λθ も適応的にスケーリングされる")
print("→ 勾配の大きいパラメータは正則化効果が弱まる")
print("")
print("-" * 60)
print("")
print("【AdamW (Decoupled Weight Decay)】")
print("")
print("1. 勾配を計算: g = ∇L（正則化なし）")
print("2. 1次モーメント: m = β₁m + (1-β₁)g")
print("3. 2次モーメント: v = β₂v + (1-β₂)g²")
print("4. 更新: θ = θ - η * m̂ / √v̂ - ηλθ")
print("")
print("→ Weight Decay は Adam の更新とは独立")
print("→ すべてのパラメータに均一に正則化が適用される")

In [ ]:
# 簡単なシミュレーションで違いを確認

def adam_l2(grad_fn, start, lr=0.1, lambda_reg=0.1, beta1=0.9, beta2=0.999, n_steps=100):
    """Adam + L2正則化"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    m = np.zeros_like(pos)
    v = np.zeros_like(pos)
    eps = 1e-8
    
    for t in range(1, n_steps + 1):
        grad = grad_fn(pos) + lambda_reg * pos  # L2正則化を勾配に含める
        
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad ** 2
        
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        
        pos = pos - lr * m_hat / (np.sqrt(v_hat) + eps)
        path.append(pos.copy())
    
    return np.array(path)


def adamw(grad_fn, start, lr=0.1, lambda_reg=0.1, beta1=0.9, beta2=0.999, n_steps=100):
    """AdamW (Decoupled Weight Decay)"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    m = np.zeros_like(pos)
    v = np.zeros_like(pos)
    eps = 1e-8
    
    for t in range(1, n_steps + 1):
        grad = grad_fn(pos)  # 正則化なしの勾配
        
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad ** 2
        
        m_hat = m / (1 - beta1 ** t)
        v_hat = v / (1 - beta2 ** t)
        
        pos = pos - lr * m_hat / (np.sqrt(v_hat) + eps) - lr * lambda_reg * pos  # Weight Decay を別途適用
        path.append(pos.copy())
    
    return np.array(path)


# テスト関数
def test_loss_grad(pos):
    return np.array([2*pos[0], 20*pos[1]])  # 異方性のある勾配


start = np.array([5.0, 5.0])
path_adam_l2 = adam_l2(test_loss_grad, start, lr=0.1, lambda_reg=0.1, n_steps=100)
path_adamw = adamw(test_loss_grad, start, lr=0.1, lambda_reg=0.1, n_steps=100)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# パラメータの推移
axes[0].plot(path_adam_l2[:, 0], label='Adam+L2 (w₁)')
axes[0].plot(path_adam_l2[:, 1], label='Adam+L2 (w₂)')
axes[0].plot(path_adamw[:, 0], '--', label='AdamW (w₁)')
axes[0].plot(path_adamw[:, 1], '--', label='AdamW (w₂)')
axes[0].set_xlabel('ステップ')
axes[0].set_ylabel('パラメータ値')
axes[0].set_title('Adam+L2 vs AdamW')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# パラメータ空間での軌跡
axes[1].plot(path_adam_l2[:, 0], path_adam_l2[:, 1], 'b.-', label='Adam+L2')
axes[1].plot(path_adamw[:, 0], path_adamw[:, 1], 'r.-', label='AdamW')
axes[1].scatter([0], [0], color='green', s=100, marker='*', zorder=5)
axes[1].set_xlabel('w₁')
axes[1].set_ylabel('w₂')
axes[1].set_title('パラメータ空間での軌跡')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"【最終パラメータ】")
print(f"Adam+L2: ({path_adam_l2[-1, 0]:.4f}, {path_adam_l2[-1, 1]:.4f})")
print(f"AdamW:   ({path_adamw[-1, 0]:.4f}, {path_adamw[-1, 1]:.4f})")

---

## 5. Elastic Net

### 5.1 定義

L1とL2を組み合わせた正則化：

$$
L_{\text{Elastic}} = L_{\text{data}} + \lambda_1 \|\theta\|_1 + \lambda_2 \|\theta\|_2^2
$$

または、混合比率 $\alpha$ を使用：

$$
L_{\text{Elastic}} = L_{\text{data}} + \lambda \left( \alpha \|\theta\|_1 + \frac{1-\alpha}{2} \|\theta\|_2^2 \right)
$$

In [ ]:
# Elastic Netの等高線

def elastic_net_penalty(w1, w2, alpha=0.5):
    l1 = np.abs(w1) + np.abs(w2)
    l2 = w1**2 + w2**2
    return alpha * l1 + (1 - alpha) * l2 / 2

alphas = [0, 0.5, 1]
titles = ['L2のみ (α=0)', 'Elastic Net (α=0.5)', 'L1のみ (α=1)']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

w = np.linspace(-2, 2, 200)
W1, W2 = np.meshgrid(w, w)

for ax, alpha, title in zip(axes, alphas, titles):
    Z = elastic_net_penalty(W1, W2, alpha)
    ax.contour(W1, W2, Z, levels=20, cmap='viridis')
    ax.set_xlabel('w₁')
    ax.set_ylabel('w₂')
    ax.set_title(title)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【Elastic Netの利点】")
print("- L1のスパース性を維持しつつ")
print("- L2により相関する特徴をグループとして選択")
print("- 特徴数 > サンプル数の場合でも安定")

---

## 6. Dropout

### 6.1 アイデア

訓練中にランダムにニューロンを無効化（出力を0に）します。

- **訓練時**: 確率 $p$ でニューロンをドロップ
- **推論時**: すべてのニューロンを使用（出力を $(1-p)$ でスケール、または訓練時に $1/(1-p)$ でスケール）

In [ ]:
class Dropout:
    """
    Dropout Layer
    
    訓練時: p の確率でニューロンを無効化、残りを 1/(1-p) でスケール
    推論時: 何もしない（Inverted Dropout）
    """
    
    def __init__(self, p=0.5):
        self.p = p
        self.mask = None
        self.training = True
    
    def forward(self, x):
        if self.training:
            self.mask = (np.random.rand(*x.shape) > self.p).astype(float)
            return x * self.mask / (1 - self.p)  # Inverted Dropout
        else:
            return x
    
    def backward(self, dout):
        if self.training:
            return dout * self.mask / (1 - self.p)
        else:
            return dout


# Dropoutの可視化
np.random.seed(42)
x = np.random.randn(1, 10)  # 10個のニューロン

dropout = Dropout(p=0.5)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 元の活性化
axes[0].bar(range(10), x.flatten())
axes[0].set_xlabel('ニューロン')
axes[0].set_ylabel('活性化')
axes[0].set_title('元の活性化')
axes[0].grid(True, alpha=0.3)

# Dropout後（訓練時）
np.random.seed(0)
dropout.training = True
x_dropped = dropout.forward(x)
colors = ['red' if dropout.mask[0, i] == 0 else 'blue' for i in range(10)]
axes[1].bar(range(10), x_dropped.flatten(), color=colors)
axes[1].set_xlabel('ニューロン')
axes[1].set_ylabel('活性化')
axes[1].set_title('Dropout後（訓練時）\n赤=ドロップ')
axes[1].grid(True, alpha=0.3)

# 推論時
dropout.training = False
x_inference = dropout.forward(x)
axes[2].bar(range(10), x_inference.flatten())
axes[2].set_xlabel('ニューロン')
axes[2].set_ylabel('活性化')
axes[2].set_title('推論時')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"ドロップされたニューロン数: {np.sum(dropout.mask == 0)}")
print(f"元の活性化の平均: {x.mean():.4f}")
print(f"Dropout後の活性化の平均: {x_dropped[dropout.mask > 0].mean():.4f}")

### 6.2 Dropoutの正則化効果

In [ ]:
print("【Dropoutの効果】")
print("")
print("1. 共適応（Co-adaptation）の防止")
print("   - ニューロンが他の特定のニューロンに依存することを防ぐ")
print("   - より頑健な特徴を学習")
print("")
print("2. アンサンブル効果")
print("   - 各ミニバッチで異なるサブネットワークを訓練")
print("   - 推論時は全サブネットワークの平均化に相当")
print("")
print("3. ノイズ注入")
print("   - 訓練データへの過度な適合を防ぐ")
print("   - データ拡張と似た効果")
print("")
print("【典型的な使用法】")
print("- 全結合層: p = 0.5")
print("- 畳み込み層: p = 0.2-0.3（または使用しない）")
print("- 最終層の直前に配置することが多い")

---

## 7. Batch Normalization

### 7.1 定義

$$
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$
$$
y_i = \gamma \hat{x}_i + \beta
$$

- $\mu_B, \sigma_B^2$: ミニバッチの平均と分散
- $\gamma, \beta$: 学習可能なパラメータ

In [ ]:
class BatchNorm:
    """
    Batch Normalization
    """
    
    def __init__(self, n_features, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        
        # 学習可能パラメータ
        self.gamma = np.ones(n_features)
        self.beta = np.zeros(n_features)
        
        # 推論用の移動平均
        self.running_mean = np.zeros(n_features)
        self.running_var = np.ones(n_features)
        
        self.training = True
    
    def forward(self, x):
        if self.training:
            mu = x.mean(axis=0)
            var = x.var(axis=0)
            
            # 移動平均の更新
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mu
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
        else:
            mu = self.running_mean
            var = self.running_var
        
        # 正規化
        self.x_norm = (x - mu) / np.sqrt(var + self.eps)
        
        # スケールとシフト
        return self.gamma * self.x_norm + self.beta


# Batch Normの効果を可視化
np.random.seed(42)

# 異なるスケールの特徴を持つデータ
x = np.column_stack([
    np.random.randn(100) * 10 + 50,  # 大きなスケール
    np.random.randn(100) * 0.1 + 0.5,  # 小さなスケール
])

bn = BatchNorm(2)
x_normalized = bn.forward(x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 正規化前
axes[0].scatter(x[:, 0], x[:, 1], alpha=0.5)
axes[0].set_xlabel('特徴1')
axes[0].set_ylabel('特徴2')
axes[0].set_title(f'正規化前\n特徴1: μ={x[:, 0].mean():.1f}, σ={x[:, 0].std():.1f}\n特徴2: μ={x[:, 1].mean():.2f}, σ={x[:, 1].std():.2f}')
axes[0].grid(True, alpha=0.3)

# 正規化後
axes[1].scatter(x_normalized[:, 0], x_normalized[:, 1], alpha=0.5)
axes[1].set_xlabel('特徴1')
axes[1].set_ylabel('特徴2')
axes[1].set_title(f'正規化後\n特徴1: μ={x_normalized[:, 0].mean():.4f}, σ={x_normalized[:, 0].std():.4f}\n特徴2: μ={x_normalized[:, 1].mean():.4f}, σ={x_normalized[:, 1].std():.4f}')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("【Batch Normalizationの効果】")
print("")
print("1. Internal Covariate Shift の軽減")
print("   - 各層の入力分布を安定化")
print("   - 学習の高速化")
print("")
print("2. 正則化効果")
print("   - ミニバッチごとに異なる正規化パラメータ")
print("   - ノイズ注入と同様の効果")
print("")
print("3. より大きな学習率の使用が可能")
print("   - 勾配が安定するため")
print("")
print("4. 初期化への依存が減少")
print("   - 入力が正規化されるため")

---

## 8. 演習問題

### 演習 8.1: L1正則化の近接勾配法

近接勾配法（Proximal Gradient Method）を使ってL1正則化付き回帰を実装してください。

In [ ]:
# 演習 8.1: 解答欄

def proximal_gradient_l1(X, y, lambda_reg, lr=0.01, max_iter=1000):
    """
    近接勾配法によるLasso回帰
    
    1. 勾配ステップ: z = w - lr * ∇L(w)
    2. 近接演算子: w = prox_{λ||·||₁}(z) = soft_threshold(z, lr*λ)
    """
    # TODO: 実装
    pass

### 演習 8.2: Dropout の勾配実装

Dropout層の backward メソッドを完成させてください。

In [ ]:
# 演習 8.2: 解答欄

# TODO: Dropout.backward() をテスト

---

## 9. まとめと次のステップ

### このノートブックで学んだこと

1. **L2正則化**: 重みを小さく保つ、滑らかな制約

2. **L1正則化**: スパースな解を生成、特徴選択

3. **Weight Decay vs L2**:
   - SGDでは同等
   - Adamでは異なる（AdamWが推奨）

4. **Dropout**: ランダムなニューロン無効化による正則化

5. **Batch Normalization**: 分布の正規化と正則化効果

### 次のノートブック（115: 高度な最適化テクニック）への橋渡し

より高度な最適化テクニック：

- Gradient Clipping
- SAM（Sharpness-Aware Minimization）
- LARS/LAMB

---

## 参考文献

1. Srivastava, N., et al. (2014). Dropout: A simple way to prevent neural networks from overfitting. *JMLR*.
2. Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training. *ICML*.
3. Loshchilov, I., & Hutter, F. (2019). Decoupled weight decay regularization. *ICLR*.
4. Tibshirani, R. (1996). Regression shrinkage and selection via the lasso. *JRSS*.